Посмотрим, какие обрезанные изображения по ошибке не проходят этап локальной проверки качества:

In [1]:
import os

LOCAL_ERRORS_PATH = os.path.join(os.getcwd(), 'repo', 'jet-project', 'local_failed_images')

In [2]:
CROP_IMAGES_PATH = os.path.join(os.getcwd(), 'repo', 'jet-project', 'crop_results')

In [7]:
import shutil

def collect_images(category_name):
    with open(os.path.join(LOCAL_ERRORS_PATH, category_name + '.txt')) as f:
        for line in f.readlines():
            curr_img_name = line.strip().replace('.png', '') + '.png'
            shutil.copyfile(
                os.path.join(CROP_IMAGES_PATH, curr_img_name),
                os.path.join(LOCAL_ERRORS_PATH, category_name, curr_img_name),
            )

In [8]:
names = ['too_bright', 'too_blurry', 'too_thin', 'too_small']

for name in names:
    collect_images(name)

Просмотрев изображения, которые отсеиваем на этапе локальной проверки, обнаружила 7 ошибочных изображений для признака 'crop is too thin' и 4 ошибочных изображения на 'too blurry'. Посчитаем значения метрик на этих изображениях, подкорректируем значения порогов (помним о том, что лучше пропустить дальше плохие изображения. чем отсеять хорошие!):

In [1]:
import os

LOCAL_ERRORS_PATH = os.path.join(os.getcwd(), 'repo', 'jet-project', 'local_failed_images')

blurry_errors_path = os.path.join(LOCAL_ERRORS_PATH, 'too_blurry', 'no!')
thin_errors_path = os.path.join(LOCAL_ERRORS_PATH, 'too_thin', 'no!')

### Порог для нечётких изображений

In [2]:
import scipy.ndimage as snd
import numpy as np

def sta6_optimized(gray_img: np.ndarray, conv_size: int = 0, stride: int = 1):
    image_intensity = gray_img / 255
    
    mean_kernel = np.ones((conv_size, conv_size)) / conv_size**2
    image_mean_intensity = snd.convolve(image_intensity, mean_kernel)
    image_mean_intensity = image_mean_intensity[(conv_size-1)//2:image_mean_intensity.shape[0]-conv_size//2, (conv_size-1)//2:image_mean_intensity.shape[1]-conv_size//2]
    image_intensity = image_intensity[(conv_size-1)//2:image_intensity.shape[0]-conv_size//2, (conv_size-1)//2:image_intensity.shape[1]-conv_size//2]

    image_intensity = image_intensity[::stride, ::stride]
    image_mean_intensity = image_mean_intensity[::stride, ::stride]
    
    sta6 = np.mean(np.power(image_intensity - image_mean_intensity, 2))

    return sta6

In [3]:
from tqdm.notebook import tqdm
import cv2

def count_sta6_on_images(path_to_images):
    values = []
    for i, img in tqdm(enumerate(os.listdir(path_to_images))):
        image = cv2.imread(os.path.join(path_to_images, img))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        curr_val = sta6_optimized(gray, conv_size=5)
        values.append(curr_val)
    return np.array(values)

In [4]:
count_sta6_on_images(blurry_errors_path) * 1e7

0it [00:00, ?it/s]

array([655.90125889, 651.11605682, 510.47749301, 550.60003033])

До этого называли обрезанное изображение слишком нечётким, если метрика STA6 * 10^7 была <= 688.078.
Поменяем порог на 510.477

### Порог для повёрнутых прутков

In [5]:
def size_ratio(gray_img: np.ndarray):
    height, width = gray_img.shape
    return min(height, width) / max(height, width)

def count_metric_on_images(metric, path_to_images):
    values = []
    for i, img in tqdm(enumerate(os.listdir(path_to_images))):
        image = cv2.imread(os.path.join(path_to_images, img))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        curr_val = metric(gray)
        values.append(curr_val)
    return np.array(values)

In [6]:
count_metric_on_images(size_ratio, thin_errors_path)

0it [00:00, ?it/s]

array([0.80017762, 0.95027624, 0.77767936, 0.81074169, 0.94951456,
       0.8011782 , 0.81024259])

До этого называли обрезанное изображение слишком повёрнутым, если метрика отношения высоты и ширины была < 0.811.
Поменяем порог на 0.777

После этого ещё раз запустим pipeline, посмотрим, какие изображения отсеились:

In [10]:
CROP_IMAGES_PATH = os.path.join(os.getcwd(), 'repo', 'jet-project', 'crop_results')

In [11]:
import shutil

def collect_images(category_name):
    with open(os.path.join(LOCAL_ERRORS_PATH, category_name[:-1] + '.txt')) as f:
        for line in f.readlines():
            curr_img_name = line.strip().replace('.png', '') + '.png'
            shutil.copyfile(
                os.path.join(CROP_IMAGES_PATH, curr_img_name),
                os.path.join(LOCAL_ERRORS_PATH, category_name, curr_img_name),
            )

In [12]:
names = ['too_bright1', 'too_blurry1', 'too_thin1', 'too_small1']

for name in names:
    collect_images(name)

Стало лучше!)